# Tree ensembles

<div style="text-align: right"><a>por </a><a href="https://www.linkedin.com/in/sheriff-data/" target="_blank">Manuel López Sheriff</a></div>

In [ ]:
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV

Los modelos ensemble en ML son aquellos que:

 * utilizan varios *weak learners* para
 * construir un modelo promedio que en general se comporta mejor que cada uno de sus componentes individuales

## Bagging

Bagging (Bootstrap Aggregation) es una herramienta de Machine Learning utilizada para:

 * mejorar la estabilidad de un algoritmo (robusto ante pequeños cambios en los datos)
 * reducir el overfitting

 

Consiste en, dado un conjunto de datos original $D$:
 1. construir diferentes conjuntos de datos $D_i$ a partir de $D$, extrayendo muestras con reemplazo (bootstraping)
 2. construir un modelo para cada conjunto de datos $D_i$
 3. para finalmente promediar las predicciones en la fase de testing (agregación)

<img width=600 src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Ensemble_Bagging.svg/440px-Ensemble_Bagging.svg.png">

## Random Forest

Random Forest aplica la lógica Bagging para construir varios árboles de decisión

<img width=600 src="https://cdn.analyticsvidhya.com/wp-content/uploads/2020/02/rfc_vs_dt1.png">

## Ejercicio

Entrena un RandomForestRegressor sobre wine_quality

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("./datasets/wine_quality.csv")

In [ ]:
df.shape

### Train test split

In [ ]:
target = "quality"

In [ ]:
X = df.drop(target, axis=1)
y = df[target]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

#### Regresión lineal

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lin = LinearRegression()

In [ ]:
lin.fit(X_train, y_train)

In [ ]:
lin_test_score = mean_squared_error(
    y_pred=lin.predict(X_test),
    y_true=y_test,
)

In [ ]:
print(f"The test score with linear regression is {lin_test_score.round(3)}")

#### Decision tree regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
tree = DecisionTreeRegressor(max_depth=6, random_state=666)

In [ ]:
tree.fit(X_train, y_train)

In [ ]:
tree_test_score = mean_squared_error(
    y_pred=tree.predict(X_test),
    y_true=y_test
)

In [ ]:
print(f"The test score with decision tree is {tree_test_score.round(3)}")

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(15, 10))
plot_tree(tree, feature_names=df.columns[:-1], filled=True);

In [ ]:
fig.savefig("wineee.svg")

#### Random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestRegressor

<img width=600 src="https://cdn.analyticsvidhya.com/wp-content/uploads/2020/02/rfc_vs_dt1.png">

 * n_estimators: número de árboles
 * max_depth: profundidad máxima de cada árbol
 * max_features: número de variables a considerar en cada división de cada árbol

In [ ]:
rf = RandomForestRegressor(n_estimators=10, max_features=0.8, max_depth=6)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
rf_test_score = mean_squared_error(
    y_pred=rf.predict(X_test),
    y_true=y_test
)

In [ ]:
print(f"The test score with random forest is {rf_test_score.round(3)}")

In [ ]:
fig = plt.figure()

for i, tree in enumerate(rf.estimators_):
    plot_tree(tree, feature_names=X_train.columns)
    fig.savefig(f"arbol_{i}.svg")

En general, RandomForest funciona mejor que el árbol de decisión

## Boosting

Boosting es otra técnica de ensemble para crear una colección de modelos. Pero en este caso:

 * los modelos se construyen secuencialmente

 * los primeros modelos ajustan modelos sencillos a los datos

 * el modelo sucesivo tiene en cuenta los errores cometidos por el modelo anterior

<img width=600 src="https://iq.opengenus.org/content/images/2020/01/boosted-trees-process.png">

## Gradient Boosting

Gradient Boosting aplica la lógica Boosting (aparte de Bagging) para construir varios árboles de decisión

Va entrenando varios **árboles de forma consecutiva** y, en cada paso

 * pondera las muestras de datos de forma diferente
 * para centrarse en los datos más difíciles de predecir

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gb = GradientBoostingRegressor(n_estimators=500, max_features=1, max_depth=10)

In [ ]:
gb.fit(X_train, y_train)

In [ ]:
gb_test_score = mean_squared_error(
    y_pred=gb.predict(X_test),
    y_true=y_test
)

In [ ]:
print(f"The test score with gradient boosting is {gb_test_score.round(3)}")

In [ ]:
n = 10

In [ ]:
preds = pd.DataFrame({
    "real": y_test[:n], 
    "lin": lin.predict(X_test[:n]), 
    "tree": tree.predict(X_test[:n]), 
    "gb": gb.predict(X_test[:n])
})

In [ ]:
preds.round(3)

En efecto, gradient boosting normalmente obtiene resultados mucho mejores!

Todavía podemos hacer GridSearchCV para encontrar los mejores hiperparámetros, y tal vez mejorar los resultados

## NOTA

 * Los Random Forest pueden ser entrenados de forma **paralela**: si construyes 1000 árboles, puedes hacerlo en diferentes máquinas
 * Gradient Boosting **no** puede entrenarse en paralelo: los árboles se construyen secuencialmente

## Guardar / Exprotar un modelo

In [ ]:
gb

In [ ]:
import pickle

In [ ]:
# save the model to disk
pickle.dump(gb, open("gb_wine_q.pkl", 'wb'))

In [ ]:
# load the model from disk
model = pickle.load(open("gb_wine_q.pkl", 'rb'))

In [ ]:
model.predict(X_test)